In [207]:
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('./')
from planar_ising.planar_ising.inference_and_sampling import InferenceAndSampling
from planar_ising.planar_ising.planar_ising_model_generator import PlanarIsingModelGenerator
from planar_ising.planar_graph.planar_graph_edges import PlanarGraphEdges
from planar_ising.planar_graph.planar_graph_generator import PlanarGraphGenerator

from triangular_lattice2 import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [264]:
Lph,T = 22,22

q=2
beta = np.log((q**2+1)/q)
L = int(Lph/2)+(int(Lph/2)%2)
p = 1-(int(Lph/2)%2)

In [271]:
basisx,basisy,sets = trnglr_lattice(L,T,p)
adjmatrix = allOR(sets)
jmatrix,E0 = model_couplings(sets,G=10) 
database = list_edges(adjmatrix,basisx,basisy)
#visualize(database,basisx,basisy,adjmatrix,jmatrix)  

In [272]:
#S1 = partition_function_method(jmatrix,adjmatrix,E0,beta)
#print("Partition function (direct):",S1)
#print("Cellular automaton:",S2)

In [274]:
#jmatrix = random_couplings(adjmatrix)

#jmatrix = jmatrix+E0*np.eye(len(jmatrix))/len(jmatrix)
jmatrix_planar = -beta*jmatrix
ising_model = PlanarIsingModelGenerator.generate_triang_ising_model(database, jmatrix_planar)
ising_inference = InferenceAndSampling(ising_model)
ising_inference.prepare()
log_partition_function = ising_inference.compute_logpf()

S = cellular_automaton_method(L=Lph,tmax=T)
print('Exact:',S)
print('Apprx:', -log_partition_function+beta*E0-np.log(0.5))

Exact: 4.841591862705236
Apprx: 4.843202246813972
